In [ ]:
from fastai.vision.all import *
import torchvision.models as models
import matplotlib.pyplot as plt
print("cuda.is_available: {}".format(torch.cuda.is_available()))

In [ ]:
def label_func(x):
    return x.parent.name

def load_data(path, size=224):
    fnames = get_image_files(path)
    item_tfms = [ToTensor()]
    if size is not None:
        item_tfms.append(Resize(size))
    dls = ImageDataLoaders.from_path_func(
        path,
        fnames,
        label_func,
        item_tfms=item_tfms,
        seed=1234,
        bs=64,
        #batch_tfms=aug_transforms(pad_mode="zeros", mult=2, min_scale=0.5),
    )
    return dls
    
path = "dogs/train/"

dls = load_data(path)

In [ ]:
dls.show_batch()

## Train classifier for dog breeds

In [ ]:
learn = cnn_learner(dls, models.resnext50_32x4d, cut=-1, pretrained=True, metrics=accuracy)

In [ ]:
lr_min = learn.lr_find()
print(lr_min.valley)

In [ ]:
# learn.fit_one_cycle(5, 0.002)
learn.fit_one_cycle(5, lr_min.valley)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
# interp.plot_confusion_matrix()

In [ ]:
# interp.plot_top_losses(10, nrows=2)

In [ ]:
learn.export("exported_resnext50_32x4d.pickle")

In [ ]:
learn.save("saved_resnext50_32x4d")